# Purpose:
- To query lims data.
- Until CO docDB search and metadata are well established (e.g., which sessions are drifting gratings?)

# Pre-requisites:
- brain_observatory_qc - updated version (7/10/2024; from_lims_update branch)
- For comparison: AllenSDK
# Misc:
- Copied from jinho_data_analysis/pilots/231106_GCaMP8s_vs_8m_from_lims.ipynb
- Add z-drift metric from mouse-seeks QC (using mongoDB)

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import h5py
from brain_observatory_qc.data_access import from_lims

from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache as bpc

from pymongo import MongoClient
mongo = MongoClient("flaskapp.corp.alleninstitute.org", 27017)


c:\Users\jinho.kim\Anaconda3\envs\allenvisb\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\jinho.kim\Anaconda3\envs\allenvisb\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\jinho.kim\Anaconda3\envs\allenvisb\lib\site-packages\numpy\.libs\libopenblas.gk7gx5keq4f6uyo3p26ulgbqyhgqo7j4.gfortran-win_amd64.dll
c:\Users\jinho.kim\Anaconda3\envs\allenvisb\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
cache = bpc.from_lims()
table = cache.get_ophys_experiment_table(passed_only=False)

# Helper functions

In [ ]:
# helper function
def get_session_info_per_group_of_mice(mouse_ids, gcamp_type):
    experiment_table = pd.DataFrame()
    problem_experiments = []
    for mouse_id in mouse_ids:
        mouse_data = from_lims.get_imaging_ids_for_mouse_id(mouse_id)
        if len(mouse_data) > 0:
            ophys_experiment_ids = mouse_data.ophys_experiment_id.values
            for ophys_experiment_id in ophys_experiment_ids: 
                try: 
                    expt_info = from_lims.get_general_info_for_ophys_experiment_id(ophys_experiment_id)
                    if len(expt_info) > 0:
                        expt_info = expt_info.iloc[[0]]
                        genotype = from_lims.get_genotype_for_ophys_experiment_id(ophys_experiment_id)
                        expt_info["mouse_id"] = mouse_id # expt_info only contains `donor_id` and `specimen_id`, need to add mouse_id here
                        expt_info["full_genotype"] = genotype.full_genotype.values[0] # base function doesnt get genotype, add it
                        experiment_table = pd.concat([experiment_table, expt_info])
                except: 
                    # print('problem for mouse_id: ', mouse_id, ', expt_id: ', ophys_experiment_id)
                    problem_experiments.append(ophys_experiment_id)
    experiment_table['gcamp'] = gcamp_type
    return experiment_table, problem_experiments


def get_session_info_per_group_of_sessions(session_ids, mouse_ids, gcamp_type):
    experiment_table = pd.DataFrame()
    problem_experiments = []
    for session_id, mouse_id in zip(session_ids, mouse_ids):
        ophys_experiment_ids = from_lims.get_ophys_experiment_ids_for_ophys_session_id(session_id).ophys_experiment_id.values
        for ophys_experiment_id in ophys_experiment_ids: 
            try: 
                expt_info = from_lims.get_general_info_for_ophys_experiment_id(ophys_experiment_id)
                if len(expt_info) > 0:
                    expt_info = expt_info.iloc[[0]]
                    genotype = from_lims.get_genotype_for_ophys_experiment_id(ophys_experiment_id)
                    expt_info["mouse_id"] = mouse_id # expt_info only contains `donor_id` and `specimen_id`, need to add mouse_id here
                    expt_info["full_genotype"] = genotype.full_genotype.values[0] # base function doesnt get genotype, add it
                    experiment_table = pd.concat([experiment_table, expt_info])
            except: 
                # print('problem for mouse_id: ', mouse_id, ', expt_id: ', ophys_experiment_id)
                problem_experiments.append(ophys_experiment_id)
    experiment_table['gcamp'] = gcamp_type
    return experiment_table, problem_experiments


def get_mongo_client(username, password, host, port):
    client = MongoClient(f'mongodb://{username}:{password}@{host}:{port}')
    return client


def get_zdrift(opid):
    try:
        client = get_mongo_client('public', 'public_password', 'qc-sys-db', 27017)
    except:
        raise('Failed to connect to mongo')
    
    opid = int(opid)
    try:
        record = client.records.metrics.find_one({'data_id': opid})
    except:
        raise('No record found for opid: ', opid)
        
    try:
        zdrift = record['local_z_stack']['z_drift_corr_um_diff']
    except:
        zdrift = np.nan
    return zdrift

def get_intensity_drift(opid):
    try:
        client = get_mongo_client('public', 'public_password', 'qc-sys-db', 27017)
    except:
        raise('Failed to connect to mongo')
    
    opid = int(opid)
    try:
        record = client.records.metrics.find_one({'data_id': opid})
    except:
        raise('No record found for opid: ', opid)
        
    try:
        intensity_drift = record['motion_corr_physio']['percent_change_intensity']
    except:
        intensity_drift = np.nan
    return intensity_drift


def get_monitor_sync(osid):
    try:
        client = get_mongo_client('public', 'public_password', 'qc-sys-db', 27017)
    except:
        raise('Failed to connect to mongo')
    
    osid = int(osid)
    try:
        record = client.records.metrics.find_one({'data_id': osid})
    except:
        raise('No record found for osid: ', osid)
        
    try:
        monitor_sync = record['rig_sync']['display_lag']
    except:
        monitor_sync = np.nan
    return monitor_sync

In [ ]:
gcamp_table = pd.DataFrame()
problems_table = []

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_ribo_aav_local, 'ribo_aav_local')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_ribo_aav_ro, 'ribo_aav_ro')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_ribo_aav_icv, 'ribo_aav_icv')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_snap25_oi4_dox, 'snap25_oi4_dox')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_slc32a1_oi4, 'slc32a1_oi4')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_slc17a7_oi4, 'slc17a7_oi4')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_slc32a1_oi1, 'slc32a1_oi1')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_slc17a7_oi1_dox, 'slc17a7_oi1_dox')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_cux2_oi1, 'cux2_oi1')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_oi4_homo, 'oi4_homo')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_cux2_ai195, 'cux2_ai195_omfish')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_gad2_ai195, 'gad2_ai195_omfish')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_gad2_ai210, 'gad2_ai210_omfish')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_oi1_icre, 'oi1_icre')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_sessions(osids_slc32a1_oi1_omfish, mids_slc32a1_oi1_omfish, 'gad2_oi1_omfish')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

assert gcamp_table.ophys_session_id.isna().sum() == 0
assert len(problems_table) == 0



# Using from_lims_utilities
- updated version (currently in from_lims_update branch; 07/10/2024)

# Manual log of mouse IDs
- or session ID if specific session is preferred (when there are multiple, use the one with the lowest zdrift failure planes)

In [13]:
mids_ribo_aav_local = [726087, 719363] # 719364 removed due to brain health issue
mids_ribo_aav_ro = [730929, 730932, 730933] # Note: all of these has reduced signal after some sessions
mids_ribo_aav_icv = [738331, 738332, 738333]
mids_snap25_oi4_dox = [726433, 747443, 755212, 759075]
mids_slc32a1_oi4 = [724567, 729088, 753561, 753562, 758265]
mids_slc17a7_oi4 = []
mids_slc32a1_oi1 = [687000, 693996, 692478, 687001, 754804, 757436] # need to exclude osid 1303235340, 1299462513
mids_slc17a7_oi1_dox = [733794, 750451, 750447]
mids_cux2_oi1 = [735169, 735170]
mids_oi4_homo = [741863, 741865, 741866, 759732, 759731, 759730]
mids_oi1_icre = [762818, 762819, 762821]
mids_cux2_ai195 = [641993, 624855, 635380]
mids_gad2_ai195 = [622537, 651007, 633542]
mids_gad2_ai210 = [628165, 622756, 612771]
mids_slc32a1_oi1_omfish = [729417, 721291, 719374]

osids_slc32a1_oi1_omfish = [1372248593, 1368578834, 1363460372]

remove_session_ids = [1303235340, 1299462513]
remove_acquisition_dates = ['2024-08-14', '2024-09-24']  # weird power setting

# Using from_lims_utilities
- updated version (currently in from_lims_update branch; 07/10/2024)

In [29]:
gcamp_table = pd.DataFrame()
problems_table = []

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_ribo_aav_local, 'ribo_aav_local')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

# temp_table, temp_problems = get_session_info_per_group_of_mice(mids_ribo_aav_ro, 'ribo_aav_ro')
# gcamp_table = pd.concat([gcamp_table, temp_table])
# problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_ribo_aav_icv, 'ribo_aav_icv')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_snap25_oi4_dox, 'snap25_oi4_dox')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_slc32a1_oi4, 'slc32a1_oi4')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_slc17a7_oi4, 'slc17a7_oi4')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_slc32a1_oi1, 'slc32a1_oi1')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_slc17a7_oi1_dox, 'slc17a7_oi1_dox')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_cux2_oi1, 'cux2_oi1')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_oi4_homo, 'oi4_homo')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_cux2_ai195, 'cux2_ai195_omfish')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_gad2_ai195, 'gad2_ai195_omfish')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_gad2_ai210, 'gad2_ai210_omfish')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_oi1_icre, 'oi1_icre')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_sessions(osids_slc32a1_oi1_omfish, mids_slc32a1_oi1_omfish, 'gad2_oi1_omfish')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

assert gcamp_table.ophys_session_id.isna().sum() == 0
assert len(problems_table) == 0



In [30]:
# Get sessions with target session types
target_session_types = ['STAGE_1', 'OPHYS_2_images_A_passive']
gcamp_table = gcamp_table[gcamp_table.session_type.isin(target_session_types)].copy()

# Manual removal of erraneous sessions
gcamp_table = gcamp_table[~gcamp_table.ophys_session_id.isin(remove_session_ids)].copy()

# If "gcamp" does not have "omfish", that choose the ones with 2 planes only
plane_counts = gcamp_table.groupby('ophys_session_id').count().ophys_experiment_id
pilot_osids = gcamp_table.query('gcamp.str.contains("omfish") == False').ophys_session_id.unique()
pilot_i = plane_counts.index.isin(pilot_osids)
error_session_ids = plane_counts.iloc[pilot_i].index.values[np.where(plane_counts.iloc[pilot_i].values != 2)]
print(f'Num error sessions due to num planes: {len(error_session_ids)}')
print(f'Removing {error_session_ids}')
gcamp_table = gcamp_table.query('ophys_session_id not in @error_session_ids')

# Remove those taken with weird power setting
error_power_sessions = gcamp_table[(gcamp_table.date_of_acquisition >= remove_acquisition_dates[0]) &
                          (gcamp_table.date_of_acquisition <= remove_acquisition_dates[1])].ophys_session_id.unique()
gcamp_table = gcamp_table[~gcamp_table.ophys_session_id.isin(error_power_sessions)].copy()

num_sessions = gcamp_table.ophys_session_id.nunique()
print(f'Total number of sessions: {num_sessions}')
num_sessions_pilot =  gcamp_table.query('gcamp.str.contains("omfish") == False').ophys_session_id.nunique()
print(f'Number of pilot sessions: {num_sessions_pilot}')
num_sessions_omfish =  gcamp_table.query('gcamp.str.contains("omfish") == True').ophys_session_id.nunique()
print(f'Number of omfish sessions: {num_sessions_omfish}')

Num error sessions due to num planes: 1
Removing [1.36951892e+09]
Total number of sessions: 102
Number of pilot sessions: 88
Number of omfish sessions: 14


In [31]:
# post-processing

# ids into int
id_columns = ['ophys_experiment_id', 'ophys_session_id', 'mouse_id']
gcamp_table[id_columns] = gcamp_table[id_columns].astype(int)

# sort by gcamp
gcamp_table = gcamp_table.sort_values('gcamp')

# Add zdrift QC metric (from mouse-seeks)
gcamp_table['zdrift'] = gcamp_table.ophys_experiment_id.apply(get_zdrift)

# Add monitor sync QC (from mouse-seeks)
gcamp_table['monitor_sync'] = gcamp_table.ophys_session_id.apply(get_monitor_sync)

# Remove sesions with error in zdrift calculation
error_osids = gcamp_table.query('zdrift.isna()').ophys_session_id.unique()
gcamp_table = gcamp_table[~gcamp_table.ophys_session_id.isin(error_osids)].copy()
gcamp_table['abs_zdrift'] = np.abs(gcamp_table.zdrift)

# add intensity drift
gcamp_table['intensity_drift'] = gcamp_table.ophys_experiment_id.apply(get_intensity_drift)


# Select columns to save
info_columns = ['ophys_experiment_id', 'ophys_session_id', 'gcamp', 'mouse_id', 'date_of_acquisition', 'session_storage_directory',
                'depth', 'session_type', 'full_genotype', 'zdrift', 'abs_zdrift', 'intensity_drift', 'monitor_sync',
                'equipment_name', 'specimen_storage_directory', 'project']
gcamp_info = gcamp_table[info_columns].set_index('ophys_experiment_id').drop_duplicates()

# changing date_of_acquisition type to show up correctly in csv
gcamp_info.date_of_acquisition = gcamp_info.date_of_acquisition.dt.strftime('%Y-%m-%d_%H-%M-%S')

# session-based table for uploading
session_info_column = ['gcamp', 'full_genotype', 'mouse_id', 'date_of_acquisition', 'session_type', 'session_storage_directory', 'project']
gcamp_session_info = gcamp_info.copy().set_index('ophys_session_id', drop=True)[session_info_column].drop_duplicates()

print(len(gcamp_info))
print(len(gcamp_session_info))

278
101


In [33]:
gcamp_session_info.gcamp.unique()

array(['cux2_ai195_omfish', 'cux2_oi1', 'gad2_ai195_omfish',
       'gad2_ai210_omfish', 'gad2_oi1_omfish', 'oi4_homo', 'ribo_aav_icv',
       'ribo_aav_local', 'slc32a1_oi1', 'slc32a1_oi4', 'snap25_oi4_dox'],
      dtype=object)

In [32]:
priority_sessions = gcamp_session_info[gcamp_session_info.session_type == 'STAGE_1']
priority_sessions

,gcamp,full_genotype,mouse_id,date_of_acquisition,session_type,session_storage_directory,project
ophys_session_id,,,,,,,
1216027197,cux2_ai195_omfish,Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6...,624855,2022-10-06_10-08-07,STAGE_1,\\allen\programs\mindscope\production\vipssn\p...,omFISHCux2Meso
1211577991,cux2_ai195_omfish,Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6...,635380,2022-09-16_11-48-41,STAGE_1,\\allen\programs\mindscope\production\omfish\p...,omFISHCux2Meso
1223679413,cux2_ai195_omfish,Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6...,641993,2022-11-04_09-08-51,STAGE_1,\\allen\programs\mindscope\production\omfish\p...,omFISHCux2Meso
1400542219,cux2_oi1,Cux2-CreERT2/wt;Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IR...,735170,2024-10-18_18-44-09,STAGE_1,\\allen\programs\mindscope\production\learning...,LearningmFISHDevelopment
1398357702,cux2_oi1,Cux2-CreERT2/wt;Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IR...,735170,2024-10-04_16-58-55,STAGE_1,\\allen\programs\mindscope\production\learning...,LearningmFISHDevelopment
1234186728,gad2_ai195_omfish,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT...,651007,2022-12-20_17-05-43,STAGE_1,\\allen\programs\mindscope\production\omfish\p...,omFISHGad2Meso
1180074253,gad2_ai195_omfish,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT...,622537,2022-05-26_09-11-59,STAGE_1,\\allen\programs\mindscope\production\omfish\p...,omFISHGad2Meso
1198804955,gad2_ai195_omfish,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT...,633542,2022-08-15_14-14-14,STAGE_1,\\allen\programs\mindscope\production\omfish\p...,omFISHGad2Meso
1177120686,gad2_ai210_omfish,Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai210(TIT...,622756,2022-05-13_18-27-54,STAGE_1,\\allen\programs\mindscope\production\omfish\p...,omFISHGad2Meso


In [39]:
priority_sort = ['snap25_oi4_dox', 'ribo_aav_icv', 'ribo_aav_local', 'oi4_homo',
                 'slc32a1_oi1', 'slc32a1_oi4', 'gad2_ai195_omfish', 'gad2_ai210_omfish',
                 'cux2_oi1', 'cux2_ai195_omfish']
sorted_priority_sessions = pd.DataFrame()
for sort_gcamp in priority_sort:
    temp_table = priority_sessions.query('gcamp == @sort_gcamp')
    sorted_priority_sessions = pd.concat([sorted_priority_sessions, temp_table])

In [41]:
print(len(sorted_priority_sessions))


54


In [46]:
sorted_priority_planes = gcamp_info[gcamp_info.ophys_session_id.isin(sorted_priority_sessions.index.values)]

In [47]:
save_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pilots\GCaMP8')
plane_save_fn = save_dir / 'gcamp_pilot_plane_info_241121_priority.csv'
sorted_priority_planes.to_csv(plane_save_fn)

session_save_fn = save_dir / 'gcamp_pilot_session_info_241121_priority.csv'
sorted_priority_sessions.to_csv(session_save_fn)

In [50]:
len(sorted_priority_sessions)//10

5

In [64]:
# Saving them to have 10 sessions in each file, with a specific columns for csv
num_sessions = 10
partitions = np.arange(0, len(sorted_priority_sessions), 10)
if partitions[-1] != len(sorted_priority_sessions):
    partitions = np.append(partitions, len(sorted_priority_sessions))
for i, (start, end) in enumerate(zip(partitions[:-1], partitions[1:])):
    temp_table = sorted_priority_sessions.iloc[start:end]
    temp_table = temp_table.reset_index()[['session_storage_directory']]
    temp_table.rename(columns={'session_storage_directory': 'lims_directory'}, inplace=True)
    temp_table['capsule_id'] = ' 2d3ec30d-a0ed-48b6-be82-03036a221810'
    temp_table['mount'] = 'ophys_mount'
    save_fn = save_dir / f'gcamp_pilot_upload_241121_priority_{i+1}.csv'
    temp_table.to_csv(save_fn, index=False)


In [61]:
temp_table

,lims_directory,capsule_id,mount
0,\\allen\programs\mindscope\production\learning...,2d3ec30d-a0ed-48b6-be82-03036a221810,ophys_mount
1,\\allen\programs\mindscope\production\vipssn\p...,2d3ec30d-a0ed-48b6-be82-03036a221810,ophys_mount
2,\\allen\programs\mindscope\production\omfish\p...,2d3ec30d-a0ed-48b6-be82-03036a221810,ophys_mount
3,\\allen\programs\mindscope\production\omfish\p...,2d3ec30d-a0ed-48b6-be82-03036a221810,ophys_mount


# Saving the result

In [8]:
save_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pilots\GCaMP8')
plane_save_fn = save_dir / 'gcamp_pilot_plane_info_241121.csv'
gcamp_info.to_csv(plane_save_fn)

session_save_fn = save_dir / 'gcamp_pilot_session_info_241121.csv'
gcamp_session_info.to_csv(session_save_fn)

# Chekcing the result
- e.g., for prioritization

In [9]:
gcamp_session_info.groupby('session_type').size()

session_type
OPHYS_2_images_A_passive    62
STAGE_1                     75
dtype: int64

In [11]:
gcamp_session_info.date_of_acquisition < '2024-08-14'

ophys_session_id
1216027197     True
1223679413     True
1211577991     True
1392941765    False
1396304884    False
              ...  
1364914325     True
1365173010     True
1400196735    False
1399862153    False
1399456896    False
Name: date_of_acquisition, Length: 137, dtype: bool

# Archive

In [2]:
mouse_id = 721291
mouse_data = from_lims.get_imaging_ids_for_mouse_id(mouse_id)


In [46]:
mouse_data.ophys_experiment_id.nunique()

184

In [64]:
oeids = mouse_data.ophys_experiment_id.values
osids = mouse_data.ophys_session_id.values
print(len(oeids))

184


In [76]:
len(mouse_data.ophys_experiment_id.values)

184

In [75]:
len(mouse_data.ophys_experiment_id.unique())

184

In [66]:
for i in range(len(mouse_data)):
    oeid = mouse_data.iloc[i].ophys_experiment_id
    osid = mouse_data.iloc[i].ophys_session_id
    expt_info = from_lims.get_general_info_for_ophys_experiment_id(oeid)
    if expt_info.ophys_session_id.values[0] != osid:
        print(f"oeid {oeid} has wrong osid {osid}")


In [61]:
expt_info = from_lims.get_general_info_for_ophys_experiment_id(oeids[0])
expt_info

,ophys_experiment_id,ophys_session_id,behavior_session_id,foraging_id,ophys_container_id,supercontainer_id,experiment_workflow_state,session_workflow_state,container_workflow_state,specimen_id,...,targeted_structure,depth,equipment_name,project,experiment_storage_directory,behavior_storage_directory,session_storage_directory,container_storage_directory,supercontainer_storage_directory,specimen_storage_directory
0,1369643468,1369500192,1369515390,0f328c8a-7005-4c63-830c-e7b6b7eb0190,1378754319,None,passed,uploaded,holding,1333456325,...,VISp,80,MESO.2,omFISHGad2Meso,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,None,\\allen\programs\mindscope\production\learning...


In [62]:
expt_info.ophys_experiment_id.values[0] == oeids[0]

True

In [65]:
expt_info.ophys_session_id.values[0] == osids[0]

True

In [79]:
genotype = from_lims.get_genotype_for_ophys_experiment_id(oeid)
genotype

,full_genotype
0,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...


In [91]:
sorted_oeids = np.sort(temp_table.ophys_experiment_id.values)
sorted_oeids[np.where(np.diff(sorted_oeids)==0)[0]+1]

array([1348159920, 1348170445, 1348170469, 1363153132, 1363887797,
       1363887803, 1364562814, 1364815955, 1365108573], dtype=int64)

In [93]:
temp_table[temp_table.ophys_experiment_id == 1348159920].drop_duplicates()

,ophys_experiment_id,ophys_session_id,behavior_session_id,foraging_id,ophys_container_id,supercontainer_id,experiment_workflow_state,session_workflow_state,container_workflow_state,specimen_id,...,project,experiment_storage_directory,behavior_storage_directory,session_storage_directory,container_storage_directory,supercontainer_storage_directory,specimen_storage_directory,mouse_id,full_genotype,gcamp
0,1348159920,1347238498,1347248821,f65a9dad-c5b8-4a8f-b7ba-3e842538a69a,1378752764,None,failed,uploaded,holding,1333456325,...,LearningmFISHTask1A,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,None,\\allen\programs\mindscope\production\learning...,721291,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,roicat
1,1348159920,1347238498,1347248821,f65a9dad-c5b8-4a8f-b7ba-3e842538a69a,1378752761,None,failed,uploaded,holding,1333456325,...,LearningmFISHTask1A,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,None,\\allen\programs\mindscope\production\learning...,721291,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,roicat


In [99]:
expt_info = from_lims.get_general_info_for_ophys_experiment_id(1348159920)
expt_info

,ophys_experiment_id,ophys_session_id,behavior_session_id,foraging_id,ophys_container_id,supercontainer_id,experiment_workflow_state,session_workflow_state,container_workflow_state,specimen_id,...,targeted_structure,depth,equipment_name,project,experiment_storage_directory,behavior_storage_directory,session_storage_directory,container_storage_directory,supercontainer_storage_directory,specimen_storage_directory
0,1348159920,1347238498,1347248821,f65a9dad-c5b8-4a8f-b7ba-3e842538a69a,1378752764,None,failed,uploaded,holding,1333456325,...,VISp,288,MESO.2,LearningmFISHTask1A,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,None,\\allen\programs\mindscope\production\learning...
1,1348159920,1347238498,1347248821,f65a9dad-c5b8-4a8f-b7ba-3e842538a69a,1378752761,None,failed,uploaded,holding,1333456325,...,VISp,288,MESO.2,LearningmFISHTask1A,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,None,\\allen\programs\mindscope\production\learning...


In [111]:
expt_info.iloc[[0]]

,ophys_experiment_id,ophys_session_id,behavior_session_id,foraging_id,ophys_container_id,supercontainer_id,experiment_workflow_state,session_workflow_state,container_workflow_state,specimen_id,...,targeted_structure,depth,equipment_name,project,experiment_storage_directory,behavior_storage_directory,session_storage_directory,container_storage_directory,supercontainer_storage_directory,specimen_storage_directory
0,1348159920,1347238498,1347248821,f65a9dad-c5b8-4a8f-b7ba-3e842538a69a,1378752764,None,failed,uploaded,holding,1333456325,...,VISp,288,MESO.2,LearningmFISHTask1A,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,None,\\allen\programs\mindscope\production\learning...


In [4]:
client = get_mongo_client('public', 'public_password', 'qc-sys-db', 27017)

In [5]:
opid = 1348159920
record = client.records.metrics.find_one({'data_id': opid})

In [7]:
record['motion_corr_physio']['percent_change_intensity']

-0.7535618863772511

In [6]:
record

{'_id': ObjectId('662e5af4f24d47b9452fa1b5'),
 'data_id': 1348159920,
 'lims_ophys_experiment': {'id': 1348159920,
  'name': '240423_721291_VISp_-1087_-273_288',
  'storage_directory': '/allen/programs/mindscope/production/learning/prod0/specimen_1333456325/ophys_session_1347238498/ophys_experiment_1348159920/',
  'depth': 288,
  'area': 'VISp',
  'workflow_state': 'qc',
  'rig': 'MESO.2',
  'calculated_depth': 288.0,
  'ophys_session': {'id': 1347238498,
   'storage_directory': '/allen/programs/mindscope/production/learning/prod0/specimen_1333456325/ophys_session_1347238498/'},
  'ophys_primary_image': '1348159920.h5',
  'ophys_z_stack_local_image': '1348159920_z_stack_local.h5',
  'averaged_surface_image': '1348159920_surface.tif',
  'averaged_depth_image': '1348159920_depth.tif',
  'maximum_intensity_projection_image': '1348159920_suite2p_maximum_projection.png',
  'ophys_primary_image_id': 1348164070,
  'target_match': False,
  'measured_area': None,
  'radius_um': None,
  'target_

# Checking mice from lims

In [ ]:
cache = bpc.from_lims()
table = cache.get_ophys_experiment_table(passed_only=False)

In [ ]:
table.project_code.unique()

array(['VisualBehaviorMultiscope', 'LearningmFISHDevelopment', 'U01BFCT',
       'OpenScopeDendriteCoupling', 'VisualBehavior',
       'VisBIntTestDatacube', 'omFISHSstMeso', 'omFISHGad2Meso',
       'VisualBehaviorTask1B', 'MultiscopeSignalNoise', 'omFISHRbp4Meso',
       'LearningmFISHTask1A', 'TaskTrainedNetworksMultiscope',
       'VisualBehaviorMultiscope4areasx2d', 'omFISHVipMeso',
       'VipAxonalV1Phase1', 'MesoscopeDevelopment', 'omFISHCux2Meso',
       'VisualBehaviorIntegrationTest', 'VisualBehaviorDevelopment',
       'OPHSx'], dtype=object)

In [ ]:
table.query('project_code == "omFISHGad2Meso"').full_genotype.unique()

array(['Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg/wt',
       'Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai210(TITL-GC7f-ICF-IRES-tTA2)-hyg/wt',
       'Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2)/wt'],
      dtype=object)

In [ ]:
table.query('project_code == "omFISHGad2Meso"').session_type.unique()

array(['STAGE_0', 'STAGE_1', 'OPHYS_2_images_A_passive'], dtype=object)

In [ ]:
table.query('project_code == "omFISHGad2Meso" and session_type == "STAGE_1"').full_genotype.unique()

array(['Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg/wt',
       'Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai210(TITL-GC7f-ICF-IRES-tTA2)-hyg/wt',
       'Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2)/wt'],
      dtype=object)

In [ ]:
table.query('project_code == "omFISHGad2Meso" and session_type == "STAGE_1" and \
             full_genotype == "Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2)/wt"')[['ophys_session_id', 'mouse_id']].drop_duplicates()

,ophys_session_id,mouse_id
ophys_experiment_id,,
1372467501,1372248593,729417
1372152666,1371951858,729417
1373065804,1372837442,729417
1368469044,1368311851,721291
1368702337,1368578834,721291
1369433942,1369312751,721291
1363363110,1363264958,719374
1363805469,1363460372,719374
1363853630,1363667121,719374


In [ ]:
table.query('project_code == "omFISHGad2Meso" and session_type == "STAGE_1"')[['full_genotype', 'ophys_session_id', 'mouse_id']].drop_duplicates().groupby('full_genotype').count()

,ophys_session_id,mouse_id
full_genotype,,
Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg/wt,38,38
Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai210(TITL-GC7f-ICF-IRES-tTA2)-hyg/wt,7,7
Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2)/wt,12,12


In [ ]:
table.query('project_code == "omFISHGad2Meso" and session_type == "STAGE_1" and \
             full_genotype == "Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai210(TITL-GC7f-ICF-IRES-tTA2)-hyg/wt"')[['ophys_session_id', 'mouse_id']].drop_duplicates()

,ophys_session_id,mouse_id
ophys_experiment_id,,
1191004722,1190915552,628165
1159170595,1159034997,611810
1177166743,1177120686,622756
1175399782,1175313136,616982
1166227159,1165883141,612771
1165994390,1165809470,612480
1164681596,1164524372,612480


In [ ]:
table.query('project_code == "omFISHGad2Meso" and session_type == "STAGE_1" and \
             full_genotype == "Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg/wt"')[['ophys_session_id', 'mouse_id']].drop_duplicates()

,ophys_session_id,mouse_id
ophys_experiment_id,,
1341293536,1341130936,717453
1342182536,1342087168,717453
1342650706,1342536034,717453
1180204496,1180074253,622537
1181945242,1181800386,622866
1191748747,1191651092,629589
1336456776,1336336487,710343
1336198792,1336020816,710343
1337024928,1336909632,710343


In [ ]:
table.query('project_code == "omFISHGad2Meso" and session_type == "STAGE_1"')[['full_genotype', 'mouse_id']].drop_duplicates().groupby('full_genotype').count()

,mouse_id
full_genotype,
Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg/wt,20
Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai210(TITL-GC7f-ICF-IRES-tTA2)-hyg/wt,6
Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2)/wt,4


In [ ]:
table.query('project_code == "omFISHCux2Meso" and session_type == "STAGE_1"')[['full_genotype', 'mouse_id']].drop_duplicates().groupby('full_genotype').count()

,mouse_id
full_genotype,
Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6f)/wt,6


In [ ]:
table.query('project_code == "omFISHCux2Meso" and session_type == "STAGE_1"')[['full_genotype', 'ophys_session_id', 'mouse_id']].drop_duplicates()

,full_genotype,ophys_session_id,mouse_id
ophys_experiment_id,,,
1223843453,Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6...,1223679413,641993
1258561417,Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6...,1258384492,662168
1258859850,Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6...,1258706332,662168
1258262554,Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6...,1258091992,662168
1232915523,Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6...,1231992811,646978
1231266819,Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6...,1229863602,646978
1215499378,Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6...,1215370616,624852
1217415915,Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6...,1217152913,624852
1216210354,Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6...,1216027197,624855


In [6]:
gcamp_table.project.unique()

array(['LearningmFISHDevelopment', 'MesoscopeDevelopment',
       'omFISHCux2Meso', 'omFISHGad2Meso'], dtype=object)

In [55]:
# target_depth_pair_list = ['75_375', '175_275']


# # Get sessions with 2 planes only (others are errors or tests)
# plane_counts = gcamp_table.groupby('ophys_session_id').count().ophys_experiment_id
# correct_session_ids = plane_counts[plane_counts==2].index.values
# gcamp_table = gcamp_table[gcamp_table.ophys_session_id.isin(correct_session_ids)].copy()

# Add target depths and target depth pairs
#### Do this later during the analysis, because now we are combining 1x2 and 1x8 sessions
# depth_divider = [0, 125, 225, 325, 500]
# depth_labels = [75, 175, 275, 375]
# gcamp_table['target_depth'] = pd.cut(gcamp_table.depth, bins=depth_divider, labels=depth_labels, right=False)
# # sort by target depth
# gcamp_table = gcamp_table.sort_values('target_depth')
# gcamp_table['target_depth_pair'] = gcamp_table.groupby('ophys_session_id').target_depth.transform(lambda x: '_'.join(x.astype(str)))

# Adding code ocean info
- Not possible. Allensdk and code ocean api clash (boto3 issue)
- Instead, use Matt's code and download the resulting csv file (e.g., assets_df_v4_v5_none_missing_2024-09-19_19-59-57_metadata.csv)

In [2]:
from lamf_analysis.code_ocean.code_ocean_data_explorer import CodeOceanDataExplorer

ImportError: cannot import name 'DEFAULT_CIPHERS' from 'urllib3.util.ssl_' (c:\Users\jinho.kim\Anaconda3\envs\allenvisb\lib\site-packages\urllib3\util\ssl_.py)

# Intead, use manually curated list of valid sessions

In [16]:
lims_gcamp_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pilots\GCaMP8')
co_asset_fn = lims_gcamp_dir / 'assets_df_v4_v5_none_missing_2024-09-19_19-59-57_metadata.csv'
asset_table = pd.read_csv(co_asset_fn)


In [17]:
gcamp_session_info['co_session_name'] = gcamp_session_info.apply(lambda x: '_'.join(f'multiplane-ophys_{x.mouse_id}_{x.date_of_acquisition}'.split('_')[:3]), axis=1)
asset_table['raw_data_asset_name_short_version'] = asset_table.raw_data_asset_name.apply(lambda x: '_'.join(x.split('_')[:-1]))
# find the ones that are not in the asset table
gcamp_session_info['in_code_ocean'] = gcamp_session_info.co_session_name.isin(asset_table.raw_data_asset_name_short_version)

In [18]:
num_in_co = gcamp_session_info.in_code_ocean.sum()
print(f'{num_in_co} sessions are in code ocean')
print(f'{len(gcamp_session_info) - num_in_co} sessions need to be uploaded')


68 sessions are in code ocean
60 sessions need to be uploaded


# Saving the result

In [19]:
save_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pilots\GCaMP8')
plane_save_fn = save_dir / 'gcamp_pilot_plane_info_241016.csv'
gcamp_info.to_csv(plane_save_fn)

session_save_fn = save_dir / 'gcamp_pilot_session_info_241016.csv'
gcamp_session_info.to_csv(session_save_fn)

# Checking some info
- mostly to use in mouse-qc

In [14]:
gcamp_info.query('gcamp == "cux2_oi1"')

,ophys_session_id,gcamp,mouse_id,date_of_acquisition,session_storage_directory,target_depth,target_depth_pair,session_type,full_genotype,zdrift,abs_zdrift,monitor_sync,equipment_name,specimen_storage_directory
ophys_experiment_id,,,,,,,,,,,,,,


# From a specific set of mouse IDs


In [121]:
mouse_ids = [721291]
temp_table, temp_problems = get_session_info_per_group_of_mice(mouse_ids, 'roicat')
print(len(temp_table))
assert (temp_table.groupby('ophys_experiment_id').count().values[:,0]==1).all()

In [126]:
# Changing IDs to integers
new_temp_table = temp_table.copy()
column_names_to_int = ['ophys_session_id', 'ophys_experiment_id', 'mouse_id', 'depth']
for cn in column_names_to_int:
    new_temp_table[cn] = new_temp_table[cn].astype(int)
print(len(new_temp_table))

# Add zdrift QC metric (from mouse-seeks)
new_temp_table['zdrift'] = new_temp_table.ophys_experiment_id.apply(get_zdrift)
print(len(new_temp_table))

# Remove sesions with error in zdrift calculation
error_osids = new_temp_table.query('zdrift.isna()').ophys_session_id.unique()
if len(error_osids) > 0:
    print(f'Remove {len(error_osids)} sessions with error in zdrift calculation')
    new_temp_table = new_temp_table[~new_temp_table.ophys_session_id.isin(error_osids)].copy()
new_temp_table['abs_zdrift'] = np.abs(new_temp_table.zdrift)
print(len(new_temp_table))

# Select columns to save
info_columns = ['ophys_experiment_id', 'ophys_session_id', 'gcamp', 'mouse_id',
                'depth', 'targeted_structure',
                'date_of_acquisition', 'session_storage_directory', 
                'session_type', 'full_genotype', 'zdrift', 'abs_zdrift',
                'equipment_name', 'specimen_storage_directory']
new_temp_table = new_temp_table[info_columns].set_index('ophys_experiment_id').drop_duplicates()
print(len(new_temp_table))

# changing date_of_acquisition type to show up correctly in csv
new_temp_table.date_of_acquisition = new_temp_table.date_of_acquisition.dt.strftime('%Y-%m-%d_%H-%M-%S')

assert len(new_temp_table) == len(temp_table)

184
184
184
184


In [128]:
save_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pilots\GCaMP8')
save_fn = save_dir / 'vgat_roicat_data_240808.csv'
new_temp_table.to_csv(save_fn)

## Testing monitor delay vs rig

In [3]:
load_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pilots\GCaMP8')
load_fn = load_dir / 'gcamp_pilot_data_240805.csv'
gcamp_info = pd.read_csv(load_fn, index_col=0)
gcamp_info.columns

Index(['ophys_session_id', 'gcamp', 'mouse_id', 'date_of_acquisition',
       'session_storage_directory', 'target_depth', 'target_depth_pair',
       'session_type', 'full_genotype', 'zdrift', 'abs_zdrift', 'monitor_sync',
       'equipment_name', 'specimen_storage_directory'],
      dtype='object')

In [4]:
gcamp_info.equipment_name.unique()

array(['MESO.2', 'MESO.1'], dtype=object)

In [10]:
gcamp_info.groupby('equipment_name')['monitor_sync'].unique().values

array([array([0.03645847, 0.03648283, 0.03650328, 0.03681988, 0.03651563,
              0.03651628, 0.03661471, 0.03646689, 0.03647455, 0.0364918 ,
              0.0364981 , 0.03650023, 0.03650128, 0.03650327, 0.03649358,
              0.04126666, 0.03653039, 0.03654316, 0.0365312 , 0.03652664,
              0.03652338])                                               ,
       array([1.03692077, 1.03682575, 1.03683722, 1.03684717, 1.03687059,
              1.03682622, 1.03694229, 1.03684363, 1.03683578, 1.0368919 ,
              1.03680525, 1.03685398, 1.0368621 , 1.03692569, 1.03684124,
              1.03684748, 1.03685351, 1.03687068, 1.03687475, 1.03683948,
              1.03674684, 1.03686376, 1.03697532, 1.03681341, 1.03682252,
              1.03690759, 1.0368589 , 1.03684613, 0.03589889, 0.03589909,
              0.03591069, 0.03595037, 0.03591569, 0.03588723, 0.03594081,
              0.03595261, 0.03592552, 0.03595338, 0.03594145, 0.03588683,
              1.03684704, 1.03684778,

In [17]:
gcamp_info.query('monitor_sync > 0.04 and monitor_sync < 0.05')

,ophys_session_id,gcamp,mouse_id,date_of_acquisition,session_storage_directory,target_depth,target_depth_pair,session_type,full_genotype,zdrift,abs_zdrift,monitor_sync,equipment_name,specimen_storage_directory
ophys_experiment_id,,,,,,,,,,,,,,
1367808840,1367710111,slc32a1_oi4,724567,2024-05-20_19-00-32,\\allen\programs\mindscope\production\learning...,375,75_375,OPHYS_2_images_A_passive,Slc32a1-IRES-Cre/wt;Oi4(TIT2L-jGCaMP8s-RiboL1-...,6.75,6.75,0.041267,MESO.1,\\allen\programs\mindscope\production\learning...
1367808838,1367710111,slc32a1_oi4,724567,2024-05-20_19-00-32,\\allen\programs\mindscope\production\learning...,75,75_375,OPHYS_2_images_A_passive,Slc32a1-IRES-Cre/wt;Oi4(TIT2L-jGCaMP8s-RiboL1-...,12.00,12.00,0.041267,MESO.1,\\allen\programs\mindscope\production\learning...


In [13]:
gcamp_info.query('equipment_name == "MESO.2"').sort_values('date_of_acquisition').monitor_sync.values

array([0.03595338, 0.03595338, 0.03591569, 0.03591569, 0.03594081,
       0.03594081, 0.03595037, 0.03595037, 0.03594145, 0.03594145,
       0.03595261, 0.03595261, 0.03591069, 0.03591069, 0.03592552,
       0.03592552, 0.03588723, 0.03588723, 0.03588683, 0.03588683,
       0.03589909, 0.03589909, 0.03589889, 0.03589889, 1.03685351,
       1.03685351, 1.03684748, 1.03684748, 1.03687059, 1.03687059,
       1.0368621 , 1.0368621 , 1.03687475, 1.03687475, 1.0368919 ,
       1.0368919 , 1.03683948, 1.03683948, 1.03683722, 1.03683722,
       1.03683578, 1.03683578, 1.03684363, 1.03684363, 1.03682575,
       1.03682575, 1.03694229, 1.03694229, 1.03685398, 1.03685398,
       1.03684124, 1.03684124, 1.03686376, 1.03686376, 1.03682311,
       1.03682311, 1.03680525, 1.03680525, 1.03683509, 1.03683509,
       1.03674684, 1.03674684, 1.03684778, 1.03684778, 1.03686567,
       1.03686567, 1.03682354, 1.03682354, 1.03684704, 1.03684704,
       1.03682498, 1.03682498, 1.03686349, 1.03686349, 1.03692

In [15]:
gcamp_info.query('equipment_name == "MESO.2" and monitor_sync < 1').date_of_acquisition.max()

'2023-11-02_16-10-37'

In [16]:
gcamp_info.query('equipment_name == "MESO.2" and monitor_sync > 1').date_of_acquisition.min()

'2024-04-11_18-36-21'

### Comparing between those in behaviorProjectCache tables

In [ ]:
# Getting results from allensdk
cache = bpc.from_lims()
table = cache.get_ophys_experiment_table(passed_only=False)

lims_path_info = pd.DataFrame()

temp = table.query('mouse_id in @mids_ribo_aav_local and session_type=="STAGE_1"').set_index('ophys_session_id')[['mouse_id', 'date_of_acquisition']].drop_duplicates()
temp['lims_path'] = temp.index.map(lambda x: from_lims.get_session_h5_filepath(x).parent)
temp['gcamp'] = 'ribo_aav_local'

lims_path_info = pd.concat([lims_path_info, temp])

temp = table.query('mouse_id in @mids_ribo_aav_ro and session_type=="STAGE_1"').set_index('ophys_session_id')[['mouse_id', 'date_of_acquisition']].drop_duplicates()
temp['lims_path'] = temp.index.map(lambda x: from_lims.get_session_h5_filepath(x).parent)
temp['gcamp'] = 'ribo_aav_ro'

lims_path_info = pd.concat([lims_path_info, temp])

temp = table.query('mouse_id in @mids_snap25_oi4_dox and session_type=="STAGE_1"').set_index('ophys_session_id')[['mouse_id', 'date_of_acquisition']].drop_duplicates()
temp['lims_path'] = temp.index.map(lambda x: from_lims.get_session_h5_filepath(x).parent)
temp['gcamp'] = 'snap25_oi4_dox'

lims_path_info = pd.concat([lims_path_info, temp])

temp = table.query('mouse_id in @mids_slc32a1_oi4 and session_type=="STAGE_1"').set_index('ophys_session_id')[['mouse_id', 'date_of_acquisition']].drop_duplicates()
temp['lims_path'] = temp.index.map(lambda x: from_lims.get_session_h5_filepath(x).parent)
temp['gcamp'] = 'slc32a1_oi4'

lims_path_info = pd.concat([lims_path_info, temp])

temp = table.query('mouse_id in @mids_slc17a7_oi1_dox and session_type=="STAGE_1"').set_index('ophys_session_id')[['mouse_id', 'date_of_acquisition']].drop_duplicates()
temp['lims_path'] = temp.index.map(lambda x: from_lims.get_session_h5_filepath(x).parent)
temp['gcamp'] = 'slc17a7_oi1_dox'

lims_path_info = pd.concat([lims_path_info, temp])

temp = table.query('mouse_id in @mids_slc32a1_oi1 and session_type=="STAGE_1"').set_index('ophys_session_id')[['mouse_id', 'date_of_acquisition']].drop_duplicates()
temp['lims_path'] = temp.index.map(lambda x: from_lims.get_session_h5_filepath(x).parent)
temp['gcamp'] = 'slc32a1_oi1'
temp = temp.drop(1303235340)

lims_path_info = pd.concat([lims_path_info, temp])

In [165]:
lims_path_info.head()

,mouse_id,date_of_acquisition,lims_path,gcamp
ophys_session_id,,,,
1363436007,726087,2024-05-01 16:00:28.374,\\allen\programs\mindscope\production\learning...,ribo_aav_local
1363660075,726087,2024-05-02 17:40:50.123,\\allen\programs\mindscope\production\learning...,ribo_aav_local
1373890769,726087,2024-06-17 16:01:47.183,\\allen\programs\mindscope\production\learning...,ribo_aav_local
1374103167,726087,2024-06-18 17:34:38.386,\\allen\programs\mindscope\production\learning...,ribo_aav_local
1347962182,719363,2024-04-26 16:40:02.748,\\allen\programs\mindscope\production\learning...,ribo_aav_local


In [158]:
len(lims_path_info)

38

In [160]:
len(np.setdiff1d(gcamp_session_info.index.values, lims_path_info.index.values))

38

In [161]:
gcamp_session_info[~gcamp_session_info.index.isin(lims_path_info.index.values)]

,gcamp,mouse_id,date_of_acquisition,session_storage_directory,target_depth_pair,session_type,full_genotype,equipment_name,session_workflow_state,specimen_storage_directory
ophys_session_id,,,,,,,,,,
1346243082,ribo_aav_local,719364,2024-04-18 20:01:34.113,\\allen\programs\mindscope\production\learning...,375_75,OPHYS_2_images_A_passive,wt/wt,MESO.2,uploaded,\\allen\programs\mindscope\production\learning...
1345660871,ribo_aav_local,719364,2024-04-16 16:34:01.738,\\allen\programs\mindscope\production\learning...,175_275,OPHYS_2_images_A_passive,wt/wt,MESO.2,uploaded,\\allen\programs\mindscope\production\learning...
1373167919,ribo_aav_local,726087,2024-06-14 15:32:17.526,\\allen\programs\mindscope\production\learning...,75_375,OPHYS_2_images_A_passive,wt/wt,MESO.2,uploaded,\\allen\programs\mindscope\production\learning...
1372856980,ribo_aav_local,726087,2024-06-13 17:47:50.392,\\allen\programs\mindscope\production\learning...,175_275,OPHYS_2_images_A_passive,wt/wt,MESO.2,uploaded,\\allen\programs\mindscope\production\learning...
1364932026,ribo_aav_local,726087,2024-05-08 17:06:03.137,\\allen\programs\mindscope\production\learning...,375_75,OPHYS_2_images_A_passive,wt/wt,MESO.2,uploaded,\\allen\programs\mindscope\production\learning...
1364667229,ribo_aav_local,726087,2024-05-07 17:49:47.971,\\allen\programs\mindscope\production\learning...,275_175,OPHYS_2_images_A_passive,wt/wt,MESO.2,uploaded,\\allen\programs\mindscope\production\learning...
1370881864,ribo_aav_local,719363,2024-06-05 15:40:39.832,\\allen\programs\mindscope\production\learning...,375_75,OPHYS_2_images_A_passive,wt/wt,MESO.2,uploaded,\\allen\programs\mindscope\production\learning...
1370649434,ribo_aav_local,719363,2024-06-04 15:41:08.712,\\allen\programs\mindscope\production\learning...,275_175,OPHYS_2_images_A_passive,wt/wt,MESO.2,uploaded,\\allen\programs\mindscope\production\learning...
1365176380,ribo_aav_local,719363,2024-05-09 15:53:25.895,\\allen\programs\mindscope\production\learning...,175_275,OPHYS_2_images_A_passive,wt/wt,MESO.2,uploaded,\\allen\programs\mindscope\production\learning...


## No noticeable difference between those included in allensdk and those not included in allensdk

# ophys QC check
- this is per plane

In [95]:
gcamp_table.abs_zdrift.unique()

array([14.25,  6.75,  8.25, 17.25,  7.5 ,  9.  , 15.75, 13.5 ,  9.75,
        3.75, 18.  , 10.5 ,  6.  ,  3.  ,  5.25, 19.5 , 11.25,  4.5 ,
       16.5 ,  2.25, 12.  ,  0.  , 12.75,  1.5 , 15.  ])

In [101]:
print(len(np.where(gcamp_table.abs_zdrift.values >= 10)[0]))
print(len(gcamp_table))
print(len(np.where(gcamp_table.abs_zdrift.values >= 10)[0])/len(gcamp_table))

53
134
0.39552238805970147


In [102]:
gcamp_table[gcamp_table.abs_zdrift.values < 10][['gcamp', 'target_depth', 'session_type', 'mouse_id', 'ophys_experiment_id']]

,gcamp,target_depth,session_type,mouse_id,ophys_experiment_id
0,ribo_aav_local,275,OPHYS_2_images_A_passive,719363,1366572037
0,ribo_aav_local,275,STAGE_1,719363,1369843883
0,ribo_aav_local,275,OPHYS_2_images_A_passive,726087,1373065786
0,ribo_aav_local,275,OPHYS_2_images_A_passive,719364,1345890441
0,ribo_aav_local,275,OPHYS_2_images_A_passive,719363,1370818210
...,...,...,...,...,...
0,snap25_oi4_dox,275,STAGE_1,726433,1371295103
0,snap25_oi4_dox,275,STAGE_1,726433,1365650901
0,snap25_oi4_dox,75,STAGE_1,726433,1372153912
0,snap25_oi4_dox,375,STAGE_1,726433,1372153914


In [108]:
gcamp_table[gcamp_table.abs_zdrift.values < 10][['gcamp', 'session_type', 'target_depth', 'mouse_id']].groupby(['gcamp', 'session_type', 'target_depth']).count()


mouse_id
gcamp          session_type             target_depth          
ribo_aav_local OPHYS_2_images_A_passive 75                   2
                                        175                  3
                                        275                  5
                                        375                  3
               STAGE_1                  75                   4
                                        175                  5
                                        275                  5
                                        375                  4
ribo_aav_ro    OPHYS_2_images_A_passive 75                   0
                                        175                  0
                                        275                  0
                                        375                  0
               STAGE_1                  75                   0
                                        175                  1
                                        275                  1
                                        375                  0
slc32a1_oi1    OPHYS_2_images_A_passive 75                   3
                                        175                  2
                                        275                  2
                                        375                  4
               STAGE_1                  75                   3
                                        175                  4
                                        275                  5
                                        375                  3
slc32a1_oi4    OPHYS_2_images_A_passive 75                   0
                                        175                  2
                                        275                  3
                                        375                  2
               STAGE_1                  75                   2
                                        175                  1
                                        275                  1
                                        375                  3
snap25_oi4_dox OPHYS_2_images_A_passive 75                   0
                                        175                  0
                                        275                  1
                                        375                  1
               STAGE_1                  75                   1
                                        175                  0
                                        275                  2
                                        375                  1